In [ ]:
import os

import nibabel as nb
import numpy as np
from functions import convert_samples_to_features, edit_events_full, extract_samples
from nilearn import image

In [ ]:
# data_root = "/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2"
data_root = "/DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2"  # sim01 dir

derivatives_dir = os.path.join(data_root, "derivatives")

fmriprep_dir = os.path.join(derivatives_dir, "fmriprep23")  # fmriprep output
func_dir = os.path.join(derivatives_dir, "func_clean")  # cleaned bold images
stab_mask_dir = os.path.join(derivatives_dir, "mvpa_08_stability_bold", "stab_musicnoise_masks")  # for stability masks
output_feat_full_dir = os.path.join(derivatives_dir, "mvpa_08_stability_bold", "features_musicnoise")  # for full features

In [ ]:
# check if output directories exist and create them if not
if not os.path.exists(output_feat_full_dir):
    os.makedirs(output_feat_full_dir)

# All subject iteration

In [ ]:
from concurrent.futures import ThreadPoolExecutor

subject_list = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]
run_list = ["1", "2", "3", "4"]


def process_subject(subject):
    print(f"Processing subject {subject}")
    stab_mask_data = np.load(os.path.join(stab_mask_dir, f"sub-{subject}_STAB.npy"))
    example_func = nb.load(os.path.join(func_dir, f"sub-{subject}_ses-01_task-02a_run-1_cleaned.nii.gz"))

    # generate image
    stab_mask_img = nb.Nifti1Image(stab_mask_data, example_func.affine, example_func.header)

    # threshold the stability map
    thres = 0.1
    stab_mask_img_thres = image.threshold_img(stab_mask_img, threshold=thres, two_sided=False)
    stab_mask_img_thres_bin = image.binarize_img(stab_mask_img_thres)

    # save the thresholded stability mask
    stab_mask_img_thres_bin.to_filename(os.path.join(stab_mask_dir, f"sub-{subject}_STAB_mask.nii.gz"))

    for run in run_list:
        if subject == "18" and run == "2":
            continue

        # load cleaned image
        img_clean = nb.load(os.path.join(func_dir, f"sub-{subject}_ses-01_task-02a_run-{run}_cleaned.nii.gz"))

        # extract samples from the cleaned inside the stability mask
        samples = extract_samples(img_clean, stab_mask_img_thres_bin, subject, run)

        # load and edit the events for classification
        events_split_2 = edit_events_musicnoise(data_root, subject, run)

        # convert samples to features
        convert_samples_to_features(samples, output_feat_full_dir, events_split_2, subject, run)


with ThreadPoolExecutor(max_workers=18) as executor:
    executor.map(process_subject, subject_list)